In [21]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

In [54]:
def create_staging_vote_df():
    '''Create statging_vote_df from raw_vote_df'''

    engine = create_engine('postgresql://localhost:5432/wa_leg_raw')
    con = engine.connect()
    vote_df = pd.read_sql_query('select * from "vote_api"',con=engine)
    vote_df['bill_unique'] = vote_df['biennium'] + ' ' + vote_df['bill_id']


    def change_biennium_to_year(biennium):
        return int(biennium[0:4])
    
    def change_vote_to_int(vote):
        if vote == 'Nay':
            return 0
        if vote == 'Yea':
            return 1
        if vote == 'Excused':
            return 2
        if vote == 'Absent':
            return 3

    def change_agency_to_int(agency):
        if agency == 'House':
            return 0
        if agency == 'Senate':
            return 1

        
    vote_df['year'] = vote_df['biennium'].apply(change_biennium_to_year)
    vote_df = vote_df.drop(['bill_id', 'biennium', 'motion'], axis=1)
    vote_df['sequence_number'] = vote_df['sequence_number'].apply(int)
    vote_df['voter_id'] = vote_df['voter_id'].apply(int)
    vote_df['vote_date'] = pd.to_datetime(vote_df['vote_date'])
    vote_df['vote'] = vote_df['vote'].apply(change_vote_to_int)
    vote_df['voting_agency'] = vote_df['voting_agency'].apply(change_agency_to_int)

    return vote_df

In [57]:
vote_df = create_staging_vote_df()

In [58]:
vote_df

,sequence_number,vote,vote_date,voter_id,voter_name,voting_agency,bill_unique,year
0,4,1,1991-02-22,7,Anderson,0,1991-92 SHB 1001,1991
1,4,1,1991-02-22,11,Appelwick,0,1991-92 SHB 1001,1991
2,4,1,1991-02-22,17,Ballard,0,1991-92 SHB 1001,1991
3,4,1,1991-02-22,23,Basich,0,1991-92 SHB 1001,1991
4,4,1,1991-02-22,32,Beck,0,1991-92 SHB 1001,1991
5,4,1,1991-02-22,34,Belcher,0,1991-92 SHB 1001,1991
6,4,2,1991-02-22,40,Betrozoff,0,1991-92 SHB 1001,1991
7,4,1,1991-02-22,45,Bowman,0,1991-92 SHB 1001,1991
8,4,0,1991-02-22,46,Braddock,0,1991-92 SHB 1001,1991
9,4,1,1991-02-22,47,Bray,0,1991-92 SHB 1001,1991


In [53]:
vote_df.to_sql('vote', con, if_exists='append', index=False)

0:300000
300000:600000
600000:900000
900000:1200000
1200000:1500000
1500000:1800000
1800000:2100000
2100000:2400000
2400000:2700000
2700000:3000000
3000000: